In [1]:
import os
import numpy as np
import torch
import torch
from torchvision import transforms,models,datasets
import torch.nn.functional as F
from rich import print
from torch import nn, optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, RandomSampler, random_split, Dataset
from tqdm import tqdm
from torchvision import transforms as T
from collections import OrderedDict

from PIL import Image
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pydicom


In [2]:
def get_resnet34():

    # Load the pre-trained ResNet-18 model
    resnet34 = models.resnet34(pretrained=False)
    new_conv1 = nn.Conv2d(
    in_channels=1,
    out_channels=resnet34.conv1.out_channels,
    kernel_size=resnet34.conv1.kernel_size,
    stride=resnet34.conv1.stride,
    padding=resnet34.conv1.padding,
    bias=resnet34.conv1.bias is not None
)

    # Copy the weights from the original conv1 layer, averaging across the input channels
    new_conv1.weight.data = resnet34.conv1.weight.data.mean(dim=1, keepdim=True)

    # Replace the original conv1 layer with the new one
    resnet34.conv1 = new_conv1


    num_ftrs = resnet34.fc.in_features  # Get the number of input features to the original fc layer
    resnet34.fc = nn.Sequential(nn.Flatten(),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 1),
                nn.Sigmoid())
#     for name, param in resnet34.named_parameters():
#         if "fc" not in name:
#             param.requires_grad = False
            
    return resnet34




In [3]:
!pip install pretrained-backbones-unet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 5.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: timm
    Found existing installation: timm 1.0.3
    Uninstalling timm-1.0.3:
      Successfully uninstalled timm-1.0.3


In [4]:
from backbones_unet.model.unet import Unet
from backbones_unet.utils.dataset import SemanticSegmentationDataset
from backbones_unet.model.losses import DiceLoss
from backbones_unet.utils.trainer import Trainer

In [5]:
num_gpus = torch.cuda.device_count()


In [6]:
model = Unet(
    backbone='resnet50', # backbone network name
    in_channels=1,            # input channels (1 for gray-scale images, 3 for RGB, etc.)
    num_classes=1,            # output channels (number of classes in your dataset)
).cuda()
# model = nn.DataParallel(model, device_ids=[i for i in range(num_gpus)])  


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


In [7]:
def get_model_mask(model, image, weights_path):
    """
    Args:
        model (torch.nn.Module): The segmentation model.
        image (PIL Image or Tensor): The input image.

    Returns:
        mask (numpy array): The mask generated by the model.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model = UNet(in_channels=1, out_channels=1).cuda()  # 3 output channels for multi-class segmentation
    model = nn.DataParallel(model, device_ids=[i for i in range(num_gpus)])  

    # Load the state dictionary
    state_dict = torch.load(weights_path)
    

    # Apply the state dictionary to the model
    model.load_state_dict(state_dict)
    
    model.eval()
    image = image.astype(np.float32)
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        mask = model(image.unsqueeze(0).cuda())
    mask = mask.squeeze().cpu().numpy()
    mask = (mask > 0.45).astype(np.uint8)  # Binarize the mask
    return mask


In [8]:
images = sorted(os.listdir("/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice"))
masks = sorted(os.listdir("/kaggle/input/segmentation-new/Segmentation_MRCNN/Mask"))


In [9]:
# DataLoader Code for U-Net-32
class ClassificationDataset(Dataset):
    def __init__(self , images, masks, model):
        self.imgs = images
        self.model = model
        self.masks = masks
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        image_path = "/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice/" + self.imgs[idx]
        weights_path = "/kaggle/input/unet-model/model_UNet32_25_ResNet50_weights.pth"
        
        image = pydicom.dcmread("/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice/" + self.imgs[idx]).pixel_array
        mask2 = pydicom.dcmread("/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice/" + self.masks[idx]).pixel_array

        
        mask = get_model_mask(model, image, weights_path)
        
        image_array_float32 = image.astype(np.float32)
        
        mask1 = mask*mask2

        # Normalize the image array to the range [0, 1]
#         image_array_normalized = (image_array_float32 - np.min(image_array_float32)) / (np.max(image_array_float32) - np.min(image_array_float32))

        segmented_image = image_array_float32*mask1
       
        label = 1
        if "healthy" in image_path:
            
            label = 0
        
        
        return segmented_image,label


In [10]:
dataset = ClassificationDataset(images, masks, model)


In [11]:
main_loader = DataLoader(dataset, batch_size=1, shuffle=True)


In [12]:
features_label = []
progress_bar = tqdm(enumerate(main_loader), total=len(main_loader), desc=f"Epoch {1}")
for i, (images,labels) in progress_bar:
        features_label.append((images.squeeze(0),labels))


Epoch 1: 100%|██████████| 1440/1440 [04:56<00:00,  4.85it/s]


In [16]:
# features_and_labels_np = np.array(features_label, dtype=object)

In [17]:
# features_and_labels_np.shape

In [18]:
# Move the model to CPU to free GPU memory
model.cpu()

# Delete the model and clear GPU memory
del model
torch.cuda.empty_cache()

In [19]:
class Classification_new(Dataset):
    def __init__(self,images):
        self.imgs = images
        
    def __len__(self):
        return len(self.imgs)

    
    def __getitem__(self, idx):
        image_path = "/kaggle/input/resnet34-eval/Result/" + self.imgs[idx]
#         weights_path = "/kaggle/input/unet-model/model_UNet32_25_ResNet50_weights.pth"
        image = pydicom.dcmread("/kaggle/input/resnet34-eval/Result/" + self.imgs[idx]).pixel_array
#         mask = get_model_mask(model, image, weights_path)

        image_array_float32 = image.astype(np.float32)

        # Normalize the image array to the range [0, 1]
#         image_array_normalized = (image_array_float32 - np.min(image_array_float32)) / (np.max(image_array_float32) - np.min(image_array_float32))

#         segmented_image = image_array_float32*mask

        label = 1
        if "healthy" in image_path:

            label = 0


        return image_array_float32,label

In [ ]:
dataset_new = Classification_new(images)

In [20]:
train_size = int(0.8 * len(features_label))
val_size = len(features_label) - train_size
train_dataset, val_dataset = random_split(features_label, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [21]:
num_gpus = torch.cuda.device_count()


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_resnet34 = get_resnet34().cuda()
model_resnet34  = nn.DataParallel(model_resnet34 , device_ids=[i for i in range(num_gpus)])  


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
device

device(type='cuda')

In [24]:
# Define the loss function and optimizer
criterion = nn.BCELoss().cuda()  # Ignore background in loss calculation
optimizer = optim.Adam(model_resnet34.parameters(), lr=5e-3)

In [25]:
num_epochs = 15

In [26]:
for epoch in range(num_epochs):
    model_resnet34.train()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

    
    for i,(images, labels) in progress_bar:
#         images,masks = images.float(),masks.float()
#         print(masks)
        images, labels = images.cuda(), labels.cuda()
#         print(labels)

#         # Forward pass
        outputs = model_resnet34(images.squeeze(0).unsqueeze(1))
#         print(labels.shape)
#         labels = labels.unsqueeze(0)
#         labels = labels.permute(1,0)
        loss = criterion(outputs, labels.float())
        if not loss.requires_grad:
            raise RuntimeError("Loss tensor does not require gradients")

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

 # Adjust learning rate at specified epochs
#     if (epoch + 1) % 20 == 0:
#         for param_group in optimizer.param_groups:
#             param_group['lr'] *= 0.05

Epoch 1: 100%|██████████| 36/36 [00:14<00:00,  2.49it/s]


Epoch [1/15], Loss: 0.5716629268394576

Epoch 2: 100%|██████████| 36/36 [00:13<00:00,  2.63it/s]


Epoch [2/15], Loss: 0.43085252079698777

Epoch 3: 100%|██████████| 36/36 [00:13<00:00,  2.65it/s]


...

Epoch [13/15], Loss: 0.02409786208793068

Epoch 14: 100%|██████████| 36/36 [00:13<00:00,  2.63it/s]


Epoch [14/15], Loss: 0.022457169129564945

Epoch 15: 100%|██████████| 36/36 [00:13<00:00,  2.63it/s]


Epoch [15/15], Loss: 0.017200687501624796

In [27]:
true_labels = []
predictions = []

In [28]:
model_resnet34.eval()

with torch.no_grad():
    progress_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {1}")
    for i,(images,labels) in progress_bar:
        images, labels = images.cuda(), labels.cuda()
        outputs = model_resnet34(images.squeeze(0).unsqueeze(1))
        preds = torch.round(outputs)  # Assuming outputs are sigmoid probabilities
#         outputs[outputs>0.5] = 1
#         outputs[outputs<0.5] = 0


        true_labels.extend(labels.cpu().numpy())
        predictions.extend(preds.cpu().numpy())


Epoch 1: 100%|██████████| 18/18 [00:01<00:00, 11.51it/s]


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [31]:
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
roc_auc = roc_auc_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC Score: {roc_auc:.4f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Accuracy: 0.6215

Precision: 1.0000

Recall: 0.5177

F1 Score: 0.6822

ROC AUC Score: 0.7588

Confusion Matrix:
[[ 62   0]
 [109 117]]

In [33]:
torch.save(model_resnet34.state_dict(), 'model_ResNet34_weights.pth')
